# Welcome to Challenge 2

In this Challenge, we will:

- Create a training script using the code from Challenge 1
- Perform unit tests on the training script
- Run an experiment from a Python Script using the ScriptRunConfig() object
- Register the model into Azure ML and log the relevant metrics

## Checkpoint 1
 Ensure that you move your training data from the data folder in Challenge 1 to the data folder found in the Challenge23 directory.

 In the below cell, we create a separate folder to contain our Experiment files, called the **driver-training** folder. This is done as delineation and keeps our process neat and organized. This will become especially helpful when we get to Challenge 3 and we create a separate folder containing files ready for deployed in the the **driver-service** folder. More on that later.

 In **driver-training**, we copy in our training data and have it as a reference for the next two Challenges.

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'driver-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/porto_seguro_safe_driver_prediction_input.csv', os.path.join(training_folder, "porto_seguro_safe_driver_prediction_input.csv"))

Checking the version of the LightGBM model. We do this to ensure that we have the latest model version. You should see version 2.3.0 being returned. 

In [ ]:
import lightgbm as lgb
"lightgbm=={}".format(lgb.__version__)

[LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a distributed gradient boosting framework developed by Microsoft. It's based on [decision tree algorithms](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/two-class-boosted-decision-tree#bkmk_research) and is typically used for classification problems with small datasets (< 1 million examples). Gradient boosting is one of the most powerful techniques for building predictive models, because it has:

- Fast training speed and higher efficiency
- Lower memory usage
- Better accuracy

If we had to break the concept down further: LightGBM is based on decision tree algorithms. Think of these algorithms as a flowchart that can be drawn out to have the structure of a tree. 

What we mean by this is that the internal nodes of the trees represent a test or a question on an attribute; each branch is the possible outcome of the question asked, and the terminal node, which is also called as the leaf node, denotes a classification label (i.e. 0 or 1, cat or dog, claim or no claim).

In a decision tree, we have several predictor variables. Depending upon these predictor variables, try to predict the so-called response variable. 

In our case, our predictor variables could be the driver's age, experience, vehicle type, health conditions, eyesight abilities, vehicle mileage, last date of vehicle service. Using these variables - which have been converted into numbers using feature engineering and data cleaning - we predict whether or not a driver will submit an insurance claim; this is our response variable.

For further reading, have a look at this post written by [Microsoft Research](https://www.microsoft.com/en-us/research/project/lightgbm/).

## Checkpoint 2: Creating the training script: train.py 

In machine learning, we require a training script to train our machine learning model. For dev/test purposes, this can be done in Jupyter notebook cells like the ones you see below. However, if we think operationally, when we want to deploy a model at scale for use at a customer site, we have to transform the machine learning model training steps into functions as part of a Python Script. Doing so enables us to build a model in the context of software engineering principles, which then in turns enables us to build applications or a CICD pipeline for model deployment.

This file defines the key functions required to train the model. 
The file can be invoked with `python train.py` for development purposes.

See the comment in the cell below for the reason for the code.

In this cell, refactor the code from the Challenge 1 notebook into a Python Script by completing the commented ## TODO sections of the train.py cell in the Challenge 2 notebook.

Open up your Challenge 1 notebook to serve as reference of what code we ran in Challenge 1.

In [ ]:
%%writefile $training_folder/train.py
# The line above uses the %%writefile command to transfer everything we write in this cell into a Python script called train.py within your training_folder.
# Import our libraries
import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm #Importing the lightgbm model package from Microsoft


def split_data(data_df):
    """Split a dataframe into training and validation datasets
       We train our model using the training dataset and then 
       tune the model using our validation dataset. Splitting our
       data, especially for supervised learning - like classification -
       ensures that we don't create a biased model and gives us a way
       to evaluate model performance on data the model hasn't "seen"
       or learnt from. 
    """
    
    ## TODO
    
    return (train_data, valid_data)


def train_model(data, parameters):
    """Train a model with the given datasets and parameters"""
    # The object returned by split_data is a tuple.
    # Access train_data with data[0] and valid_data with data[1]
    
    ## TODO
    
    return model


def get_model_metrics(model, data):
    """Construct a dictionary of metrics for the model"""
    
    ## TODO
    
    return model_metrics


def main():
    """This method invokes the training functions for development purposes"""
    
    # Read data from a file
    data_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

    # Hard code the parameters for training the model
    parameters = {
        'learning_rate': 0.02,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'sub_feature': 0.7,
        'num_leaves': 60,
        'min_data': 100,
        'min_hessian': 1,
        'verbose': 2
    }

    # Call the functions defined in this file and assign them to variables to ensure the outputs of one function can be fed as inputs into the next function.
    ## TODO
    
    # Print the resulting metrics for the model
    ## TODO
    
if __name__ == '__main__':
    main()


## Checkpoint 3

Now we need to pass our Unit Tests. This ensures that our code makes sense. We pass through dummy data to our training script and expect to pass 3 tests. 

Ensure the provided unit tests pass for train.py by calling pytest in the Terminal, which is started from the Notebook's UI. Browse the contents of the file with tests to understand what types of tests may be relevant. Refer to [Step 3 in the HOL for Challenge 2](https://github.com/tgokal/EY-MSFTAI-Workshop3/blob/master/Challenge23/HOL_Challenge_2.MD#-steps-) for help. 

## Checkpoint 4

Make the `train.py` and `test_train.py` files pass linting with `flake8` in the Terminal. You do not need to fix every single linting error here. This is for your understanding of how to produce quality code and does not impact the output and outcome. Once you have a good understnading of what the errors mean and how you would resolve them, complete and run the remaining cells of the Challenge 2 notebook.

## Checkpoint 5: Create a JSON file for the model parameters: parameters.json

This file will specify the parameters used to train the model.

We put this information into a separate file because it gives us independence and a central truth in how we play around with our variables. 

Rather than changing every occurence of a variable in a training script, we can play around with these constants from a single file.

In particular, our parameters include:

- `Learning rate`: Controls the rate at which an algorithm learns the values of the parameters.

- `Boosting type`: Type of algorithm used in technique of gradient boosting. Here, we use 'gbdt' = gradient boosting decision tree.

- `Objective`: Algorithm goal. Here, we're doing binary classification. 

- `Metric`: A score used to assess performance of our algorithm. Here, we use AUC = area under the curve. The closer to 1.00, the "better" the model performance.

- `Sub feature`: Another term for feature fraction. Selects a subset of features on each tree. Here, we set it to be 0.7 = LightGBM will select 70% of features before training each tree.

- `Number of leaves in the decision tree`: Controls the complexity of the model. More leaves increases the accuracy on the training set, but also the chance of overfitting the model to the data.

- `Min data`: Minimal number of data in 1 leaf. We use this parameter to control over-fitting. Default is typically 20 data samples per leaf.

- `Min Hessian`: Minimal sum of a value called the Hessian. Used to account for the number of observations in a lead of our split. If the value > 1 the decision tree that we create won't be that large and will be a lower complexity model.

- `Verbose`: Useful for debugging, sets the level of information returned back during model training. < 0 = Fatal only, 0 = Error only, 1 = Warnings only, > 1 = Debug

In [ ]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "learning_rate": 0.02,
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "auc",
        "sub_feature": 0.7,
        "num_leaves": 60,
        "min_data": 100,
        "min_hessian": 1,
        "verbose": 0
    }
}


## Checkpoint 6: Creating a "remote control" training script: driver_training.py
This file will be the entry script when running an Azure ML context - it calls the training script but is shaping it in the context of the Azure ML platform.
It calls the functions defined in train.py for data preparation and training, but reads parameters from a file, and logs output to the Azure ML context.  

The file can be invoked with `python driver_training.py` for development purposes.

Complete the `##TODO` sections of the `driver-training.py` file.

In [ ]:
%%writefile $training_folder/driver_training.py
# Import libraries
import argparse
from azureml.core import Run
import joblib
import json
import os
import pandas as pd
import shutil

# Import functions from train.py
from train import split_data, train_model, get_model_metrics

# Get the output folder for the model from the '--output_folder' parameter
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="outputs")
args = parser.parse_args()
output_folder = args.output_folder

# Get the experiment run context
run = Run.get_context()

# load the safe driver prediction dataset
train_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

# Load the parameters for training the model from the file
with open("parameters.json") as f:
    pars = json.load(f)
    parameters = pars["training"]

# Log each of the parameters to the run
for param_name, param_value in parameters.items():
    run.log(param_name, param_value)
    
# Use the functions imported from train.py to prepare data, train the model, and calculate the metrics
## TODO

# Log the metrics variable using run.log()

# Save the trained model to the output folder
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/porto_seguro_safe_driver_model.pkl"
joblib.dump(value=model, filename=output_path)

run.complete()

In [ ]:
## Defining an outputs folder for the model artefact

output_folder = "outputs"
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/porto_seguro_safe_driver_model.pkl"
print(output_path)

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()

## Checkpoint 7: Use ScriptRunConfig to run the script as an Experiment

Now that we've defined our model parameters, training script and then remote training script with the Azure ML context, we can submit it for training to Azure ML as an experiment. However, since we've coded up our own model, we submit this to the platform using the ScriptRunConfig object. A ScriptRunConfig packages together the configuration information needed to submit a run in Azure ML, including the script, compute target, environment, and any distributed job-specific configs.

A ScriptRunConfig object is used to configure the information necessary for submitting a training run as part of an Experiment. When a run is submitted using a ScriptRunConfig object, the submit method returns an object of type ScriptRun. Then returned ScriptRun object gives you programmatic access to information about the training run. ScriptRun is a child class of Run.

The key concept to remember is that there are different configuration objects that are used to submit an experiment, based on what kind of run you want to trigger. The type of the configuration object then informs what child class of Run you get back from the submit method. When you pass a ScriptRunConfig object in a call to Experiment's submit method, you get back a ScriptRun object. 

Complete each of the steps in separate Notebook cells.

1. First, create a [compute cluster](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python). This compute cluster will be used to train our ML model and we can programmatically create it using a code sample like the one provided in the link. 

    Name your compute cluser as `drivercluster`.

In [ ]:
# Create an compute cluster
## TODO

2. [Create an environment](https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/#in-azure-ml-sdk) and add the conda and pip packages beforing registering it into the Workspace. Use the pip, joblib, scikit-learn, pandas and lightgbm as the only pip packages. 

    **Don't just copy and paste the code from the link - think about why we need/don't need PyTorch as a channel.**
    
     Rename your environment accordingly to `driver_training_env`.

    This makes our packages as part of a reuseable software environment. When we train a model in Azure ML, it pulls a Docker image for training, referencing the YML file which contains these packages.

In [ ]:
# Create an environment

# Include this line 
conda.add_conda_package('python=3.7')

## TODO

# This will be the last line of code
env = Environment('driver_training_env')
env.python.conda_dependencies = conda
env.register(ws)

3. [Create an experiment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py), retrieve the compute target and environment before configuring and submitting your training run using the ScriptRunConfig() object on your local machine. 

    This step defines Experiment in Azure ML and retrieves the compute target, environment and submits the training run.

In [ ]:
# Create an experiment
## TODO

# Include this variable
args = "outputs"

## Stop

Switch back to your Azure ML UI and observe the training of the model in the Experiments tab. This training should take 5 mins overall and you should see a Completed green tick under the Details Tap of your Experiment Run. 

**Do not run the below cells before you have a successfully completed trained model.**

## Checkpoint 8

View the learning_rate parameter in the configuration file, execute a run against Azure ML compute to train the model and verify that the metrics are logged. Change the learning_rate value and then execute another run and see the values changed in the Azure ML service.

In [ ]:
# Print the resulting metrics
metrics = run.get_metrics()
for k, v in metrics.items():
        print(k, v)

## Checkpoint 9

Register the model in the Azure ML model repository using the provided notebook code in your Azure ML workspace - using tags to record the AUC metric in the registration so that the quality of the model is registered and not just the run.

In [ ]:
# Register the model
run.register_model(model_path='outputs/porto_seguro_safe_driver_model.pkl', model_name='porto_seguro_safe_driver_model.pkl')

## Stop

Switch back to your Azure ML UI and observe the model artefact in the Models tab.

## Coaching Questions

Once you've completed the Challenge, tag your Coach in your Team Channel and discuss the following questions with them:

- What is the benefit of separating the training code out of the notebook?

- What is the benefit of running your experiments using Azure Machine Learning?

- What are the benefits of using a model repository and tracking the metrics and other parameters?

- What is your experience in your organization/customers regarding doing things like model registration and tracking training metrics?

## Success Criteria
To have completed this Challenge, you should have:

- Have a fully functional Challenge 2 notebook (.ipynb)

- Successfully runs your experiment on Azure ML and can see the logged AUC metrics and trained model in the run results using two different parameter configuration values.

- Successfully registers the trained model and tags the model with the AUC metric.

- Demonstrate that the unit tests passs against the Python training code using pytest.

- Use flake8 to demonstrate that train.py and test_train.py conform to the PEP 8 style guide for Python code.

- Do a local run of the notebook and show the metrics associated both with the run and the registered model in Azure ML.

- Discuss the Coaching Questions above with your Coach and Team.

# <u> Welcome to Challenge 3 </u>

You just transformed the experimental notebook into a Python script that can be managed and run independently of the notebook environment. You used the script to train the model, then you used code in the notebook to register the model that the script produced.

To keep the training, registration and future steps like evaluation as easily reproducible as possible, we can encapsulate the model training and registration steps in an Azure ML pipeline which utilizes provisioned on-demand scalable Azure compute targets. The Azure compute target optimizes the time spent running training with a full set of data and can scale down to no cost when the job is done.

In order to improve the insurance application, we can use a real-time prediction of the likelihood that a driver will file a claim. To accomplish this objective, we'll be deploying the registered model as a real-time inferencing REST service using the provisioned model scoring script.

In this Challenge, we will:
- Retrieve the most recent version of the registered insurance claim prediction model.

- Create a socring script which includes an `init` function that loads the registered model and a `run` function that uses it to predict claim classifications for new driver data.

- Define the container environment that includes the Python packages required by your scoring script.

- Deploy the model as an Azure Container Instance service.

- Testing the deployed service by submitting a REST request to its endpoint and review the predictions it returns.



## Checkpoint 1

In the below cell, we'll retrieve the most recent version of the registered insurance claim prediction model.

In [ ]:
# Determine model name and version
model = ws.models['porto_seguro_safe_driver_model.pkl']
print(model.name, 'version', model.version)

Next, we're creating a folder for the web service files. This provides delineation of training and deployment. 

In [ ]:
import os

folder_name = 'driver-service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_driver.py")

## Checkpoint 2: Define the scoring script

Our scoring script is the next step. This script contains step by step instructions of what will happen when we deploy to our inference compute, in our case this will be an Azure Container Instance. What happens is this - Azure Machine Learning will create a docker image for you automatically that will package up the model, the runtime and the scoring script. This docker image will then be sent and built on the Azure Container Instance. 

Why does this matter? Often data scientists - especially at our customers - aren't familiar with the software engineer lifecycle and using technologies like Docker to deliver software as containers. Traditionally, data scientists have been doing statistical experiments - like the one we've just done - in a Jupyter Notebook, like this one. And that's it. No deployment. 

But of course, that doesn't make sense for the real-world as we put models to action. Azure Machine Learning makes it easier for data scientists to transition and learn software engineering techniques. Another element of this which you'll see later on; automatically generating a REST API endpoint for your model by using the InferenceConfig and returning a service scoring URI. 

Coming back to the below cell. 

The `init` method loads and de-serialises the model from the .pkl file and received parameters packed into the model artefact, like the ones we defined above.

The `run` method passes the parameters and data to the model for scoring and then returns the resulting predictions and probabilities.

Complete the below # TODO.

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('porto_seguro_safe_driver_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['no claim', 'claim']
    predicted_classes = []
    probabilities = [prediction for prediction in predictions]

    # TODO: 
    """Write a for-loop which appends classnames[0] to the predicted_classes
       list if the probability < 0.5, else to classnames[1]
    """
    # Return the predictions
    return (probabilities, predicted_classes)

In the cell above, you should have defined the `run` function for the scoring script which will call the model predictions and returns the probabilities and predicted classes. This is building logic so that we're interpreting the probability to what it means for the business outcome - taking the spectrum of probabilities that exist between 0 and 1 and sorting it into a binary class; the driver will make a claim vs. the driver won't make a claim.

## Checkpoint 3: Define the container environment

Here, we're creating a YAML file which contains our dependencies for the model to run. 

Why does this matter? 

When we deploy our model to Azure Container Instances, Azure Machine Learning creates a scoring image that contains your model, packages and all needed files for the model to be built and successfully predit on new data on the Container Instance. Think of it as screenshot of what software is needed for the model to work.

For this to work, the software environment which we used to train our model - all 3rd party libraries - need to be added so that when we deploy our model to Azure Container Instances, the software rebuilds in the exact same environment that was used for training and development. 

It sounds like a complex process, but all we do is specify our packages and create our YAML file which is passed into our scoring Docker image using the InferenceConfig later on. 

You just need to "point" to the packages that we used for model building and understand how we're generating the YAML file below using the CondaDependencies class.

Use this [link](https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment#in-azure-ml-sdk) as reference to complete the below #TODO, where you're adding `scikit-learn`, `lightgbm` and `pandas` dependencies for the model.

If you're familiar with creating your own Docker image, you could bring your own - creating your own Dockerfile and registering it as an environment in Azure Container Registry.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# TODO: Add the dependencies for our model (AzureML defaults is already included)


# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"driver_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

## Checkpoint 4: Configure the InferenceConfig

Here, we'll use this link to deploy the model to Azure Container Instances once we configure the InferenceConfig.

As we deploy the model using the InferenceConfig object, Azure Machine Learning creates a docker image including the model and the scoring script and then pushes this to Azure Container Registry. This image is like a screenshot of your model, dependencies, scoring script and all the necessary code for it to work on a compute instance. We take this image and deploy it as a container on Azure Container Instances.

Use this [link](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py#remarks) to get started on what you need to do to complete the # TODO below. 

**Note: the example used in the above link has a parameter mistake. Instead of the parameter `environment`, use `conda_file`**

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# TODO: Configure the scoring environment

service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.state)
print(service.get_logs())

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## Checkpoint 5: Use Webservice for 1 driver

Now that we've successfully created a service scoring URI, we need to test it out. 

In the below cell, we're submitting data - where each entry represents a feature that has been transformed into a number using a technique like one-hot encoding or generating a probability distribution - to the service endpoint as a JSON document.

Once the call to the web service is successfully, we access the predictions by indexing the returning array.

In [ ]:
import json

x_new = [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

print("Probability of claim is:", predictions[0][0])
print("Predicted class is: ", predictions[1])

## Use Webservice for multiple drivers

So, we've down it for 1 driver, now we're testing if our endpoint works for multiple drivers. In this case, we're testing for two drivers. Out of interest of time, we're not testing further, but you should always scale out and test how the model received data for multiple inputs. Typically, as it works for a number > 1, it should in theory work for all.

Same process as the above, but now we're submitting 2 feature arrays, each representing a different driver.

In [ ]:
import json

# This time our input is an array of two feature arrays
x_new = [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1],
         [4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1]]
# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
for i in range(len(predictions)):
    print("Driver {} submits {} with a probability of {}".format(i+1, predictions[1][i], round(predictions[0][i], 4)))

## Return the scoring endpoint

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

## Checkpoint 6: Submit POST Request to Webservice

Now that we've gained confidence that the endpoint can work for multiple drivers, we're going to use the endpoint but access it using a POST request, which is what developers in the Customer will do once deployed. 

If you're new to sending POST requests, read up about it [here](https://docs.microsoft.com/en-us/learn/modules/use-apis-discover-museum-art/2-what-is-api) and [here](https://www.w3schools.com/tags/ref_httpmethods.asp). 

Same process as the above, but now we're submitting 2 feature arrays, each representing a different driver.

In [ ]:
import requests
import json

x_new = [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1],[4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = (requests.post(endpoint, input_json, headers = headers)).json()

# Get the predicted classes.
for i in range(len(predictions)):
    print("Driver {} submits {} with a probability of {}".format(i+1, predictions[1][i], round(predictions[0][i], 4)))

## Coaching Questions

- What are the benefits of splitting the ML process into steps?

- What are the benefits of using the InferenceConfig object for deployment?

## Success Criteria

- Define the `run` function for the scoring script which will call the model predictions and returns the probabilities and predicted classes.

- Add the model dependencies into the container environment.

- Define the InferenceConfig() which is used to deploy the model to Azure Container Instances.

- Successfully deploy the trained model as a REST service in Azure Container Instance and test its endpoint.

- Discuss the Coaching Questions with your Coach and Team.

